In [32]:
from sentence_transformers import SentenceTransformer
import pandas as pd
import string
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /Users/elmi/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [4]:
model = SentenceTransformer('bert-base-nli-mean-tokens')

In [6]:
df = pd.read_csv('/Users/elmi/Projects/CloudWine/data/processed/sample_10000.csv')
df.head()

,title,description,points,price,variety,region_1
0,Apolloni 2015 Estate Pinot Grigio (Willamette ...,This is a light easy-drinking style that fills...,87,16.0,Pinot Grigio,Willamette Valley
1,Pemartín NV Fino Palomino (Jerez),"Mild aromas of butter, hay and sea water feed ...",89,13.0,Palomino,Jerez
2,Fabre Montmayou 2011 Phebus Estate Bottled Mal...,Jammy berry and cassis aromas come with a bit ...,88,16.0,Malbec,Río Negro Valley
3,L'Ecole No. 41 2011 Estate Grown Seven Hills V...,"A five-grape, Bordeaux-style blend, it's still...",90,50.0,Bordeaux-style Red Blend,Walla Walla Valley (WA)
4,Clos LaChance 2015 Chardonnay (Monterey County),"Squeezed lime, pear shavings and poached apple...",85,15.0,Chardonnay,Monterey County


In [7]:
df['variety_region'] = df[['variety', 'region_1']].agg('-'.join, axis=1)
corpus = df['description'].tolist()

In [8]:
corpus_lower = [x.lower() for x in corpus]

In [37]:
corpus_punct = [s.translate(str.maketrans('','',string.punctuation)) for s in corpus_lower]

In [38]:
sentences = corpus_punct

In [39]:
sentence_embeddings = model.encode(sentences)

In [40]:
df

,title,description,points,price,variety,region_1,variety_region
0,Apolloni 2015 Estate Pinot Grigio (Willamette ...,This is a light easy-drinking style that fills...,87,16.0,Pinot Grigio,Willamette Valley,Pinot Grigio-Willamette Valley
1,Pemartín NV Fino Palomino (Jerez),"Mild aromas of butter, hay and sea water feed ...",89,13.0,Palomino,Jerez,Palomino-Jerez
2,Fabre Montmayou 2011 Phebus Estate Bottled Mal...,Jammy berry and cassis aromas come with a bit ...,88,16.0,Malbec,Río Negro Valley,Malbec-Río Negro Valley
3,L'Ecole No. 41 2011 Estate Grown Seven Hills V...,"A five-grape, Bordeaux-style blend, it's still...",90,50.0,Bordeaux-style Red Blend,Walla Walla Valley (WA),Bordeaux-style Red Blend-Walla Walla Valley (WA)
4,Clos LaChance 2015 Chardonnay (Monterey County),"Squeezed lime, pear shavings and poached apple...",85,15.0,Chardonnay,Monterey County,Chardonnay-Monterey County
...,...,...,...,...,...,...,...
8341,Joullian 2011 Family Reserve Syrah (Carmel Val...,The reductive scent of rubber meets with deep ...,85,35.0,Syrah,Carmel Valley,Syrah-Carmel Valley
8342,Hawley 2015 Ponzo Vineyard Zinfandel (Russian ...,Thick and rich with tobacco and dark chocolate...,90,32.0,Zinfandel,Russian River Valley,Zinfandel-Russian River Valley
8343,Proulx 2013 Rosé Mission (Paso Robles),This wine is made from California's historic M...,85,14.0,Mission,Paso Robles,Mission-Paso Robles
8344,High Pass 2015 Zauberberg Pinot Noir (Willamet...,This well-made wine is powered by lush red and...,91,35.0,Pinot Noir,Willamette Valley,Pinot Noir-Willamette Valley


In [41]:
top_labels = ['Cabernet Sauvignon-Napa Valley','Nebbiolo-Barolo',
            'Pinot Noir-Willamette Valley','Pinot Noir-Russian River Valley',
            'Champagne Blend-Champagne']

In [65]:
df_subset = df[df['variety_region'].isin(top_labels)].reset_index(drop=True)

In [69]:
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity

In [67]:
x = np.array(sentence_embeddings)[df_subset.index.tolist()]

In [70]:
labels = df_subset['variety_region'].unique()
for l in labels:
    idx = df_subset[df_subset['variety_region'] == l].index.to_numpy()
    x_cluster = x[idx]
    cluster_mean = np.mean(x_cluster, axis=0)
    out = np.mean(cosine_similarity([cluster_mean], x_cluster).flatten())
    print(l, out)

Pinot Noir-Russian River Valley 0.8582928
Champagne Blend-Champagne 0.8642991
Nebbiolo-Barolo 0.8548904
Pinot Noir-Willamette Valley 0.8744622
Cabernet Sauvignon-Napa Valley 0.8620437


In [72]:
import pickle

In [73]:
with open('./bert_model.pkl', "wb") as pickleFile:
    pickle.dump(model, pickleFile)

/opt/anaconda3/envs/cloudwine/lib/python3.8/site-packages/torch/storage.py:34: FutureWarning: pickle support for Storage will be removed in 1.5. Use `torch.save` instead
  warnings.warn("pickle support for Storage will be removed in 1.5. Use `torch.save` instead", FutureWarning)


In [74]:
SentenceTransformer()

TypeError: stat: path should be string, bytes, os.PathLike or integer, not SentenceTransformer

In [75]:
loaded = pickle.load(open('./bert_model.pkl', 'rb'))

['this is a light easydrinking style that fills the quaffable white wine bill quite nicely its lemony and tart with a slice of orange peel for extra interest',
 'mild aromas of butter hay and sea water feed into a crisp fresh palate thats punchy and solid lightly salted briny whitefruit flavors are classic for a fino while this offers hints of vanilla and white pepper on the back end']

In [78]:
loaded.encode(sentences[:2])

[array([-9.51691926e-01,  1.97701573e-01, -6.45826280e-01, -1.11334391e-01,
        -3.06275636e-01, -1.11499369e+00, -6.01661742e-01,  1.38630066e-02,
         9.36021209e-01, -3.92155766e-01,  8.02055359e-01,  9.78839457e-01,
         4.27253135e-02, -1.75825164e-01,  2.39358693e-01,  4.21118706e-01,
         5.11493325e-01, -1.05376080e-01,  5.43285459e-02,  6.81944937e-02,
        -4.01655287e-02, -7.09712923e-01, -3.91211838e-01, -6.30229831e-01,
         4.24937159e-01, -1.97814718e-01, -1.85893014e-01,  6.80619836e-01,
         2.26089716e-01,  5.88223100e-01,  3.08697056e-02,  1.53716075e+00,
        -3.11390966e-01, -4.82717514e-01, -8.87468159e-02,  1.19907212e+00,
         1.04338363e-01,  1.22299656e-01,  1.60691723e-01,  6.40079677e-01,
         6.20002151e-01, -1.07922637e+00,  6.96291387e-01, -5.63883185e-01,
        -4.65472698e-01, -7.22084403e-01, -6.82679772e-01,  5.02485096e-01,
        -3.38992253e-02, -1.11321241e-01,  1.09511793e+00, -6.57893479e-01,
         2.9